## Árboles de decisión con imágenes

Anteriormente, pese a que lo mejor para el tratamiento de imágenes ya habíamos comentado que no son estos métodos sino los basados en redes neuronales, habíamos realizado una buena clasificación de imágenes de números que reflejaban en trazo de sus autores en una maravillosa definición de 64 pixels en escala de grises.

Sin embargo, después llegamos a probar la clasificación de imágenes un poco más complejas y la calidad de la clasificación no llegaba a ser tan buena (aunque tampoco era precisamente mala, teniendo en cuenta los datos y en nº de clases). Pues es la hora de la verdad, vamos a comprobar si nuestro nuevo árbol de decisión es capaz de hacer frente al clasificador logístico en la identificación de tipos de Pokémon.

Al igual que vimos en su día, tendremos las funciones más usadas y el código para obtener un dataset de 4800 columnas, donde cada una de ellas hace referencia a la intensidad y el canal de cada pixel.

1. Crea un modelo basado en árbol de decisión para diferenciar el tipo de los pokemon:

In [26]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [27]:
def image_to_data(img_path):
    # img_path es la dirección de la imagen:
    return plt.imread(img_path).reshape(1, 4800)

def data_to_image(img_array):
    # img_array es un array con los datos de la imagen
    return plt.imshow(np.reshape(img_array, (30, 40, 4)), cmap=plt.cm.gray)
    

Nos creamos un DataFrame para convertir todas las imágenes en datos:

In [28]:
df = pd.DataFrame(columns=list(range(0, 4800)))

for i in range(1, 152):
    new_reg = pd.DataFrame(plt.imread(f"../../2-Logistic Regression/data/pkmn/{i}.png").reshape(1, 4800), index=[i])
    df = df.append(new_reg)


In [29]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
1,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,...,0.807843,0.0,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,0.807843,0.0
2,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0
3,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0
4,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0
5,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0


In [30]:
# Y un diccionario de tipos que nos vendrá bien en el futuro:
id_to_type = {0: 'Grass', 1: 'Fire', 2: 'Water', 3: 'Bug', 4: 'Normal', 5: 'Poison', 6: 'Electric', 7: 'Ground', 8: 'Fairy', 9: 'Fighting',
              10: 'Psychic', 11: 'Rock', 12: 'Ghost', 13: 'Ice', 14: 'Dragon'}

type_to_id = {'Grass': 0, 'Fire': 1, 'Water': 2, 'Bug': 3, 'Normal': 4, 'Poison': 5, 'Electric': 6, 'Ground': 7, 'Fairy': 8, 'Fighting': 9,
              'Psychic': 10, 'Rock': 11, 'Ghost': 12, 'Ice': 13, 'Dragon': 14}

1. Lee el DataFrame de los pokemon original y pégale su tipo 1 a cada imagen de pkmn:

In [31]:
df_poke = pd.read_csv("../../../../data/Pokemon.csv", encoding='latin1', index_col='#')
df_poke

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...
147,Dratini,Dragon,NaN,300,41,64,45,50,50,50,1,False
148,Dragonair,Dragon,NaN,420,61,84,65,70,70,70,2,False
149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,3,False


In [32]:
df = df.join(df_poke['Type 1'])
df

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,Type 1
1,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,...,0.0,0.352941,0.54902,0.807843,0.0,0.352941,0.54902,0.807843,0.0,Grass
2,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Grass
3,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Grass
4,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Fire
5,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Fire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Dragon
148,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Dragon
149,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Dragon
150,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,...,0.0,1.000000,1.00000,1.000000,0.0,1.000000,1.00000,1.000000,0.0,Psychic


Buscando darle otra vuelta, vamos a idear otra forma de atacar este problema. Si investigamos el dataset, nos podemos dar cuenta de que hay un total de 31 intensidades distintas por cada canal (Red/Green/Blue/Alpha), por lo que vamos a ver si contando los puntos de cada canal podemos sacar algo.

La estructura de cada punto de la imagen, según lo que leemos es [R G B A], siendo después convertidos en repeticiones de esa estructura de forma secuencial hasta terminar la imagen de 30x40 pixels.

Se puede conseguir obtener, por cada registro, los diferentes valores para R, para G, para B y para A. Para hacerlo, deberás leer cada registro de manera independiente y centrarte en los 4800 valores que tiene. Tendrás que separar los que pertenezcan a R, los que pertenezcan a G, lo mismo para los de B y también para los de A. Podrías hacer esta primera separación mediante slicing saltando cierto número de pasos y empezando en zonas diferentes para cada color.

Después, una vez tengas una lista (por ejemplo) con cada uno, podrías convertirla a Series y utilizar el value_counts. Tras ello, podrías convertir ese series en un DataFrame cuyas columnas fueran las filas del Series. Podrías probar a convertir en un DataFrame directamente ese series y trasponerlo (haciendo ``df.T``).

Repite esto para cada uno de los 4 posibles valores y concatena los df que te dan (recuerda cambiar los nombres para hacerlo. Podrías renombrar cada columna con <R/G/B/A> + \<valor\>

Una vez lo tengas, podrías meterlo dentro de un bucle for e iterar a lo largo de todos los registros, haciendo algo semejante a lo que tenemos en el for del principio de este ejercicio, pero cambiando lo que acabamos de comentar.


2. Replica, a partir de lo que tienes a continuación, la creación y entrenamiento del modelo. Prueba a ver si mejoramos los resultados en predicción. ¿Obtenemos mejores resultados? ¿Son estables con variaciones como podría ser la semilla? ¿Se te ocurre algún método de poder mejorarlo? ¿Tiene alguna ventaja este método respecto al anterior?


Generamos un nuevo DataFrame (new_df) con todas las columnas posibles:

In [33]:
new_df = pd.DataFrame(columns=['R_0.0',
 'G_0.0',
 'B_0.0',
 'A_0.0',
 'R_0.062745101749897',
 'G_0.062745101749897',
 'B_0.062745101749897',
 'A_0.062745101749897',
 'R_0.09803921729326248',
 'G_0.09803921729326248',
 'B_0.09803921729326248',
 'A_0.09803921729326248',
 'R_0.12941177189350128',
 'G_0.12941177189350128',
 'B_0.12941177189350128',
 'A_0.12941177189350128',
 'R_0.16078431904315948',
 'G_0.16078431904315948',
 'B_0.16078431904315948',
 'A_0.16078431904315948',
 'R_0.1921568661928177',
 'G_0.1921568661928177',
 'B_0.1921568661928177',
 'A_0.1921568661928177',
 'R_0.22745098173618317',
 'G_0.22745098173618317',
 'B_0.22745098173618317',
 'A_0.22745098173618317',
 'R_0.25882354378700256',
 'G_0.25882354378700256',
 'B_0.25882354378700256',
 'A_0.25882354378700256',
 'R_0.29019609093666077',
 'G_0.29019609093666077',
 'B_0.29019609093666077',
 'A_0.29019609093666077',
 'R_0.32156863808631897',
 'G_0.32156863808631897',
 'B_0.32156863808631897',
 'A_0.32156863808631897',
 'R_0.3529411852359772',
 'G_0.3529411852359772',
 'B_0.3529411852359772',
 'A_0.3529411852359772',
 'R_0.38823530077934265',
 'G_0.38823530077934265',
 'B_0.38823530077934265',
 'A_0.38823530077934265',
 'R_0.41960784792900085',
 'G_0.41960784792900085',
 'B_0.41960784792900085',
 'A_0.41960784792900085',
 'R_0.45098039507865906',
 'G_0.45098039507865906',
 'B_0.45098039507865906',
 'A_0.45098039507865906',
 'R_0.48235294222831726',
 'G_0.48235294222831726',
 'B_0.48235294222831726',
 'A_0.48235294222831726',
 'R_0.5176470875740051',
 'G_0.5176470875740051',
 'B_0.5176470875740051',
 'A_0.5176470875740051',
 'R_0.5490196347236633',
 'G_0.5490196347236633',
 'B_0.5490196347236633',
 'A_0.5490196347236633',
 'R_0.5803921818733215',
 'G_0.5803921818733215',
 'B_0.5803921818733215',
 'A_0.5803921818733215',
 'R_0.6117647290229797',
 'G_0.6117647290229797',
 'B_0.6117647290229797',
 'A_0.6117647290229797',
 'R_0.6470588445663452',
 'G_0.6470588445663452',
 'B_0.6470588445663452',
 'A_0.6470588445663452',
 'R_0.6784313917160034',
 'G_0.6784313917160034',
 'B_0.6784313917160034',
 'A_0.6784313917160034',
 'R_0.7098039388656616',
 'G_0.7098039388656616',
 'B_0.7098039388656616',
 'A_0.7098039388656616',
 'R_0.7411764860153198',
 'G_0.7411764860153198',
 'B_0.7411764860153198',
 'A_0.7411764860153198',
 'R_0.772549033164978',
 'G_0.772549033164978',
 'B_0.772549033164978',
 'A_0.772549033164978',
 'R_0.8078431487083435',
 'G_0.8078431487083435',
 'B_0.8078431487083435',
 'A_0.8078431487083435',
 'R_0.8392156958580017',
 'G_0.8392156958580017',
 'B_0.8392156958580017',
 'A_0.8392156958580017',
 'R_0.8705882430076599',
 'G_0.8705882430076599',
 'B_0.8705882430076599',
 'A_0.8705882430076599',
 'R_0.9019607901573181',
 'G_0.9019607901573181',
 'B_0.9019607901573181',
 'A_0.9019607901573181',
 'R_0.9372549057006836',
 'G_0.9372549057006836',
 'B_0.9372549057006836',
 'A_0.9372549057006836',
 'R_0.9686274528503418',
 'G_0.9686274528503418',
 'B_0.9686274528503418',
 'A_0.9686274528503418',
 'R_1.0',
 'G_1.0',
 'B_1.0',
 'A_1.0'])

new_df

,R_0.0,G_0.0,B_0.0,A_0.0,R_0.062745101749897,G_0.062745101749897,B_0.062745101749897,A_0.062745101749897,R_0.09803921729326248,G_0.09803921729326248,...,B_0.9372549057006836,A_0.9372549057006836,R_0.9686274528503418,G_0.9686274528503418,B_0.9686274528503418,A_0.9686274528503418,R_1.0,G_1.0,B_1.0,A_1.0


In [34]:
# Cuando lo tengas diseñado, puedes formar el DataFrame completo así:
dic_color = {0: 'R', 1: 'G', 2: 'B', 3: 'A'}

for i in range(0, 151):
    df_color = pd.DataFrame([df_poke["Type 1"].iloc[i]], columns=['Type 1'])
    for color in range(0, 4):
        try:
            serie = pd.Series(df.drop("Type 1", 1).iloc[i].values[color::4])
            b = pd.DataFrame(serie.value_counts()).T
            b.columns = [f"{dic_color[color]}_{col}" for col in b.columns]
            df_color = df_color.join(b)
        except:
            print(color)
            print(i)
            print(df_color)
    new_df = new_df.append(df_color)
    


In [35]:
new_df = new_df.fillna(0).reset_index(drop=True)

In [36]:
new_df

,R_0.0,G_0.0,B_0.0,A_0.0,R_0.062745101749897,G_0.062745101749897,B_0.062745101749897,A_0.062745101749897,R_0.09803921729326248,G_0.09803921729326248,...,A_0.9372549057006836,R_0.9686274528503418,G_0.9686274528503418,B_0.9686274528503418,A_0.9686274528503418,R_1.0,G_1.0,B_1.0,A_1.0,Type 1
0,0,0,0,949,0,0,0,0,0,0,...,0,0,0,0,0,5,5,5,251,Grass
1,0,0,0,917,0,0,0,0,0,0,...,0,0,0,0,0,921,921,921,283,Grass
2,0,0,3,719,0,0,0,0,0,0,...,0,0,0,0,0,749,724,724,481,Grass
3,0,0,0,990,0,0,0,0,1,0,...,0,55,4,0,0,993,993,993,210,Fire
4,0,0,0,921,0,0,0,0,1,0,...,0,79,3,0,0,925,925,925,279,Fire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0,0,0,1008,0,0,0,0,0,0,...,0,0,0,0,0,1038,1038,1063,192,Dragon
147,0,0,0,927,0,0,0,0,0,0,...,0,0,0,0,0,964,964,989,273,Dragon
148,0,0,0,872,0,0,0,0,0,0,...,0,121,0,0,0,879,879,879,328,Dragon
149,0,0,0,918,0,0,0,0,0,0,...,0,0,0,0,0,918,918,918,282,Psychic
